In [1]:
import theano
print(theano.__version__)

0.9.0dev1.dev-fa327997673c637b16e98d726417c8c304dd7b2f


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)


In [2]:
import os
import numpy as np

from scipy import ndimage
from scipy.misc import imresize

from sklearn.utils import shuffle
from sklearn.cross_validation import StratifiedKFold
from keras.utils import np_utils

from sklearn.feature_extraction.image import extract_patches_2d

directory = './Images/'
# input image dimensions
# img_rows, img_cols = 1024, 1024
img_rows, img_cols = 128, 128

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

patch_count = 100

def load(seed=123):
    X = []
    y = []
    
    X_test = []
    y_test = []
    
    for i in [2, 4]:
        cur_direct = directory + 'Cluster ' + str(i) + '/'
        j = 0
        label = i/2-1
        print(label)
        for jpg in os.listdir(cur_direct):
            j += 1
            
            x = ndimage.imread(cur_direct + jpg)
            patches = extract_patches_2d(x, (img_rows, img_cols), patch_count)
    
            if j < 10:
                X_test.extend(patches)
                
                y_test.extend([label] * patch_count)
            else:
                X.extend(patches)
                y.extend([label] * patch_count)

    
    X = np.asarray(X)
    X_test = np.asarray(X_test)
    y = np.asarray(y)
    y_test = np.asarray(y_test)
    
    print(X.shape)
    print(y.shape)
    X = X.reshape(X.shape[0], 3, img_rows, img_cols)
    X, y = shuffle(X, y, random_state=seed)  # shuffle train data
    
    X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    
    X = X.astype('float32')
    y = y.astype('float32')
    
    X_test = X_test.astype('float32')
    y_test = y_test.astype('float32')
    return (X, y, X_test, y_test)

X, y, X_test, y_test = load()
print(X.shape)
print(y.shape)

0
1
(19800, 128, 128, 3)
(19800,)
(19800, 3, 128, 128)
(19800,)


Using Theano backend.


In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

nb_classes = 2
accuracy = []
nb_epochs = 100
batch_size = 256

X_train = X
X_test = X_test
y_train = y
y_test = y_test

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(y_train.shape, y_test.shape)

print(np.unique(y_train))
print(np.unique(y_test))
# convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_test = np_utils.to_categorical(y_test, nb_classes)
Y_train = y_train
Y_test = y_test

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(3, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)

# fit the model on the batches generated by datagen.flow()
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    samples_per_epoch=X_train.shape[0],
                    nb_epoch=nb_epochs,
                    validation_data=(X_test, Y_test),
                    nb_worker=1)

score = model.evaluate(X_test, Y_test, verbose=0)
print('Metrics', model.metrics_names)
print('Test score:', score[0])
print('Test accuracy:', score[1])
accuracy.append(score[1])
print(accuracy)

('X_train shape:', (19800, 3, 128, 128))
(19800, 'train samples')
(1800, 'test samples')
((19800,), (1800,))
[ 0.  1.]
[ 0.  1.]
Epoch 1/100
19800/19800 [==============================] - 72s - loss: 1.1900 - acc: 0.5484 - val_loss: 0.7007 - val_acc: 0.5000
Epoch 2/100
19800/19800 [==============================] - 71s - loss: 0.6510 - acc: 0.5912 - val_loss: 0.7058 - val_acc: 0.5000
Epoch 3/100
19800/19800 [==============================] - 71s - loss: 0.6239 - acc: 0.6361 - val_loss: 0.7359 - val_acc: 0.5000
Epoch 4/100
19800/19800 [==============================] - 72s - loss: 0.6480 - acc: 0.5981 - val_loss: 0.7138 - val_acc: 0.5000
Epoch 5/100
19800/19800 [==============================] - 72s - loss: 0.5238 - acc: 0.7477 - val_loss: 0.7996 - val_acc: 0.5000
Epoch 6/100
19800/19800 [==============================] - 73s - loss: 0.7229 - acc: 0.5051 - val_loss: 0.6949 - val_acc: 0.5000
Epoch 7/100
19800/19800 [==============================] - 72s - loss: 0.6934 - acc: 0.5051 - val